# Model

Radical pairs allegedly play an [essential role in avian magnetoreception](https://github.com/tretyakovmipt/RPM-toy-simulation) and can be thought of as two entangled electron spins separated in space. The pair is created between a donor and acceptor, which can be separate molecules or different parts of a bigger molecule. Initially, both donor and acceptor have a pair of electrons in a singlet state $|\uparrow\downarrow\rangle-|\downarrow\uparrow\rangle$ on the same energy level (fig A).

The first step is an excitation of one of the acceptor electrons via optical absorption, preserving the entanglement (fig B). After that, an electron from the donor is transferred to the created vacancy in the acceptor, where it has to form a singlet state with the acceptor electron (fig C). It is presumed that the remaining electrons also form a singlet state.

# Gate Model

In [1]:
from qiskit import QuantumCircuit, assemble, Aer
from math import pi, sqrt
from qiskit.visualization import plot_bloch_multivector, plot_histogram

sim = Aer.get_backend('aer_simulator') # Quantum gate simulator

First, let us test how we can create a singlet state for two spins

In [2]:
qc = QuantumCircuit(2) # Initialize a circuit with 2 qubits starting in |0>
qc.h(0) # Hadamard gate on qubit 0
qc.z(0) # Z-gate on qubit 0
qc.x(1) # NOT gate on qubit 1 putting it to state |1>
qc.cnot(0,1) # CNOT with qubit 0 as the control
qc.draw()

┌───┐┌───┐     
q_0: ┤ H ├┤ Z ├──■──
     ├───┤└───┘┌─┴─┐
q_1: ┤ X ├─────┤ X ├
     └───┘     └───┘

In [3]:
# Apply gates and show the result

qc.save_statevector()   # Tell simulator to save statevector
qobj = assemble(qc)
result = sim.run(qobj).result() # Do the simulation and return the result
out_state = result.get_statevector()
print(out_state) # Display the output state vector

Statevector([ 0.        +0.j, -0.70710678+0.j,  0.70710678+0.j,
             -0.        +0.j],
            dims=(2, 2))


It's working. Now let's do it for two pairs.

In [4]:
qc = QuantumCircuit(4)
qc.h(0)
qc.z(0)
qc.x(1)
qc.cnot(0, 1)
qc.h(2)
qc.z(2)
qc.x(3)
qc.cnot(2, 3)
qc.draw()

┌───┐┌───┐     
q_0: ┤ H ├┤ Z ├──■──
     ├───┤└───┘┌─┴─┐
q_1: ┤ X ├─────┤ X ├
     ├───┤┌───┐└───┘
q_2: ┤ H ├┤ Z ├──■──
     ├───┤└───┘┌─┴─┐
q_3: ┤ X ├─────┤ X ├
     └───┘     └───┘

These gate put the system into the state $\left(|0\rangle_{0}|1\rangle_{1} - |1\rangle_{0}|0\rangle_{1} \right] \otimes \left[|0\rangle_{2}|1\rangle_{3} - |1\rangle_{2}|0\rangle_{3} \right)$. At the end, it should become $\left(|0\rangle_{0}|1\rangle_{3} - |1\rangle_{0}|0\rangle_{3} \right] \otimes \left[|0\rangle_{2}|1\rangle_{1} - |1\rangle_{2}|0\rangle_{1} \right)$, which can be achieved by SWAP of 1 and 3.

In [5]:
qc.swap(1, 3)
qc.draw()

┌───┐┌───┐        
q_0: ┤ H ├┤ Z ├──■─────
     ├───┤└───┘┌─┴─┐   
q_1: ┤ X ├─────┤ X ├─X─
     ├───┤┌───┐└───┘ │ 
q_2: ┤ H ├┤ Z ├──■───┼─
     ├───┤└───┘┌─┴─┐ │ 
q_3: ┤ X ├─────┤ X ├─X─
     └───┘     └───┘

In [6]:
# Apply gates and show the result

qc.save_statevector()
qobj = assemble(qc)
result = sim.run(qobj).result()
out_state = result.get_statevector()
print(out_state)

Statevector([ 0. +0.j,  0. +0.j,  0. +0.j, -0.5+0.j, -0. +0.j,  0.5-0.j,
             -0. +0.j, -0. +0.j,  0. +0.j,  0. +0.j,  0.5+0.j,  0. +0.j,
             -0.5+0.j, -0. +0.j, -0. +0.j, -0. +0.j],
            dims=(2, 2, 2, 2))


Compare to the expected state

In [7]:
qc = QuantumCircuit(4)
qc.h(0)
qc.z(0)
qc.x(3)
qc.cnot(0, 3)
qc.h(2)
qc.z(2)
qc.x(1)
qc.cnot(2, 1)
qc.draw()

┌───┐┌───┐          
q_0: ┤ H ├┤ Z ├──■───────
     ├───┤└───┘  │  ┌───┐
q_1: ┤ X ├───────┼──┤ X ├
     ├───┤┌───┐  │  └─┬─┘
q_2: ┤ H ├┤ Z ├──┼────■──
     ├───┤└───┘┌─┴─┐     
q_3: ┤ X ├─────┤ X ├─────
     └───┘     └───┘

In [8]:
qc.save_statevector()
qobj = assemble(qc)
result = sim.run(qobj).result()
out_state = result.get_statevector()
print(out_state)

Statevector([ 0. +0.j,  0. +0.j,  0. +0.j, -0.5+0.j, -0. +0.j,  0.5-0.j,
             -0. +0.j, -0. +0.j,  0. +0.j,  0. +0.j,  0.5+0.j,  0. +0.j,
             -0.5+0.j, -0. +0.j, -0. +0.j, -0. +0.j],
            dims=(2, 2, 2, 2))


***
Does it mean that the electron transfer is a swap gate??